In [1]:
from spellsource.context import Context
from spellsource.behaviour import *
from spellsource.utils import simulate
from tqdm import tqdm

In [2]:
import pandas as pd
import random
import re
import collections

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
arena = pd.read_csv('cards.csv')
ctx = Context()

In [4]:
def deck_to_string(cards: List[str], deck_id: str, draft_class: str):
    result = """### {}
# Class: {}
# Format: Standard
# Year of the Raven
#
""".format(deck_id, draft_class)
    for card in cards:
        result += "# 1x (2) {}\n".format(card)
    return result

def generate_random_arena_deck(cards: pd.DataFrame, draft_class: str, deck_id: int):
    player_cards = cards[cards['Draft Class'] == draft_class]['Card Name']
    deck = [random.choice(player_cards) for i in range(30)]
    return deck_to_string(deck, deck_id, draft_class)

def get_deck_cards(deck: str):
    result = re.findall(r'# 1x \(2\) (.*)', deck)
    return result

In [14]:
desks = [generate_random_arena_deck(arena, 'Druid', i) for i in range(100)]

In [15]:
behaviour = 'GameStateValueBehaviour'
result = list(tqdm(simulate(
    behaviours=(behaviour, behaviour), 
    decks=desks,
    number=10,
    context=ctx)))

  0%|          | 14/4950 [00:36<5:01:59,  3.67s/it]


In [9]:
win_prob = collections.Counter()
occur = collections.Counter()
for matchup in result:
    for index in range(2):
        deck = desks[int(matchup['decks'][index])]
        match_result = matchup['results'][index]
        for card in set(get_deck_cards(deck)):
            win_prob[card] += match_result['WIN_RATE']
            occur[card] += 1

In [10]:
win_rate = (pd.Series(win_prob) / pd.Series(occur)).sort_values(ascending=False)

In [11]:
best_deck = deck_to_string(win_rate[:30].index, "Best Deck", "Druid")

In [13]:
list(tqdm(simulate(
    behaviours=('GameStateValueBehaviour', 'GameStateValueBehaviour'), 
    decks=[best_deck, generate_random_arena_deck(arena, 'Druid', 'Random Deck')],
    number=100,
    context=ctx)))

100%|██████████| 1/1 [00:19<00:00, 19.39s/it]


[{'decks': ['Best Deck', 'Random Deck'],
  'numberOfGames': 100,
  'results': [{'WIN_RATE': 0.79,
    'GAMES_WON': 79,
    'GAMES_LOST': 21,
    'DAMAGE_DEALT': 5216,
    'MANA_SPENT': 3774,
    'CARDS_PLAYED': 1095,
    'TURNS_TAKEN': 896,
    'ARMOR_GAINED': 119,
    'CARDS_DRAWN': 901,
    'MINIONS_PLAYED': 2085,
    'SPELLS_CAST': 50,
    'HERO_POWER_USED': 107,
    'CARDS_DISCARDED': 1},
   {'WIN_RATE': 0.21,
    'GAMES_WON': 21,
    'GAMES_LOST': 79,
    'DAMAGE_DEALT': 7654,
    'HEALING_DONE': 5,
    'MANA_SPENT': 3954,
    'CARDS_PLAYED': 1417,
    'TURNS_TAKEN': 868,
    'ARMOR_GAINED': 174,
    'CARDS_DRAWN': 869,
    'MINIONS_PLAYED': 2317,
    'SPELLS_CAST': 227,
    'HERO_POWER_USED': 171}]}]